In [1]:
import QuantLib as ql

In [16]:
class CommonVars:
    swapSettlementDays = 2
    faceAmount = 100.0
    fixedConvention = ql.Unadjusted
    compounding = ql.Continuous
    fixedFrequency = ql.Annual
    floatingFrequency = ql.Semiannual
    termStructure = ql.RelinkableYieldTermStructureHandle()
    iborIndex = ql.Euribor(ql.Period(floatingFrequency), termStructure)
    calendar = iborIndex.fixingCalendar()
    swapIndex = ql.SwapIndex("EuriborSwapIsdaFixA",ql.Period("10Y"), swapSettlementDays,
                              iborIndex.currency(), calendar,
                              ql.Period(fixedFrequency), fixedConvention,
                              iborIndex.dayCounter(), iborIndex)
    spread = 0.0
    nonnullspread = 0.003
    today = ql.Date(24,ql.April,2007)
    ql.Settings.instance().evaluationDate = today
    termStructure.linkTo(ql.FlatForward(today, 0.05, ql.Actual365Fixed()))
    pricer = ql.BlackIborCouponPricer()
    swaptionVolatilityStructure = ql.SwaptionVolatilityStructureHandle(ql.ConstantSwaptionVolatility(today, ql.NullCalendar(),ql.Following,
                                               0.2, ql.Actual365Fixed()))
    meanReversionQuote = ql.QuoteHandle(ql.SimpleQuote(0.01))
    cmspricer = ql.AnalyticHaganPricer(swaptionVolatilityStructure,
                                      ql.GFunctionFactory.Standard,
                                      meanReversionQuote)

# Consistency between fair price and fair spread

In [19]:
variables = CommonVars()

In [22]:
bondCalendar = ql.TARGET()
settlementDays = 3

In [23]:
# Fixed Underlying bond (Isin: DE0001135275 DBR 4 01/04/37)
# maturity doesn't occur on a business day

In [25]:
bondSchedule = ql.Schedule(ql.Date(4,ql.January,2005),
                      ql.Date(4,ql.January,2037),
                      ql.Period(ql.Annual), bondCalendar,
                      ql.Unadjusted, ql.Unadjusted,
                      ql.DateGeneration.Backward, False)

In [38]:
for d in bondSchedule:
    print(d)

January 4th, 2005
January 4th, 2006
January 4th, 2007
January 4th, 2008
January 4th, 2009
January 4th, 2010
January 4th, 2011
January 4th, 2012
January 4th, 2013
January 4th, 2014
January 4th, 2015
January 4th, 2016
January 4th, 2017
January 4th, 2018
January 4th, 2019
January 4th, 2020
January 4th, 2021
January 4th, 2022
January 4th, 2023
January 4th, 2024
January 4th, 2025
January 4th, 2026
January 4th, 2027
January 4th, 2028
January 4th, 2029
January 4th, 2030
January 4th, 2031
January 4th, 2032
January 4th, 2033
January 4th, 2034
January 4th, 2035
January 4th, 2036
January 4th, 2037


In [26]:
bond = ql.FixedRateBond(settlementDays, variables.faceAmount,
                      bondSchedule,
                      [ 0.04],
                      ql.ActualActual(ql.ActualActual.ISDA),
                      ql.Following,
                      100.0, ql.Date(4,ql.January,2005))

In [27]:
payFixedRate = True

In [28]:
bondPrice = 95.0

In [29]:
isPar = True

In [31]:
parAssetSwap = ql.AssetSwap(payFixedRate,
                         bond, bondPrice,
                         variables.iborIndex, variables.spread,
                         ql.Schedule(),
                         variables.iborIndex.dayCounter(),
                         isPar)

In [41]:
for cf in parAssetSwap.leg(0):
    print(cf.date(), cf.amount())

January 4th, 2008 3.9999101729171427
January 5th, 2009 4.000089827082864
January 4th, 2010 4.0000000000000036
January 4th, 2011 4.0000000000000036
January 4th, 2012 3.9999101729171427
January 4th, 2013 4.000089827082864
January 6th, 2014 4.0000000000000036
January 5th, 2015 4.0000000000000036
January 4th, 2016 3.9999101729171427
January 4th, 2017 4.000089827082864
January 4th, 2018 4.0000000000000036
January 4th, 2019 4.0000000000000036
January 6th, 2020 3.9999101729171427
January 4th, 2021 4.000089827082864
January 4th, 2022 4.0000000000000036
January 4th, 2023 4.0000000000000036
January 4th, 2024 3.9999101729171427
January 6th, 2025 4.000089827082864
January 5th, 2026 4.0000000000000036
January 4th, 2027 4.0000000000000036
January 4th, 2028 3.9999101729171427
January 4th, 2029 4.000089827082864
January 4th, 2030 4.0000000000000036
January 6th, 2031 4.0000000000000036
January 5th, 2032 3.9999101729171427
January 4th, 2033 4.000089827082864
January 4th, 2034 4.0000000000000036
January 

In [32]:
swapEngine = ql.DiscountingSwapEngine(variables.termStructure,
                              True,
                              bond.settlementDate(),
                              ql.Settings.instance().evaluationDate)

In [33]:
parAssetSwap.setPricingEngine(swapEngine)

In [42]:
for cf in parAssetSwap.leg(1):
    print(cf.date(), cf.amount())

April 27th, 2007 -3.7616438356164252
July 4th, 2007 0.9358588770158827
January 4th, 2008 2.552582336038789
July 4th, 2008 2.5244895866360695
January 5th, 2009 2.5666315972451503
July 6th, 2009 2.5244895866360695
January 4th, 2010 2.5244895866360695
July 5th, 2010 2.5244895866360695
January 4th, 2011 2.5385349992569806
July 4th, 2011 2.5104460979124887
January 4th, 2012 2.552582336038789
July 4th, 2012 2.5244895866360917
January 4th, 2013 2.552582336038811
July 4th, 2013 2.510446097912511
January 6th, 2014 2.5806827831396317
July 4th, 2014 2.4823648911033835
January 5th, 2015 2.566631597245128
July 6th, 2015 2.5244895866360695
January 4th, 2016 2.5244895866360695
July 4th, 2016 2.5244895866360473
January 4th, 2017 2.552582336038811
July 4th, 2017 2.510446097912511
January 4th, 2018 2.552582336038789
July 4th, 2018 2.510446097912511
January 4th, 2019 2.552582336038811
July 4th, 2019 2.510446097912511
January 6th, 2020 2.580682783139654
July 6th, 2020 2.5244895866360473
January 4th, 2021 

In [34]:
fairCleanPrice = parAssetSwap.fairCleanPrice()

In [36]:
fairCleanPrice

82.91948367169132

In [35]:
fairSpread = parAssetSwap.fairSpread()

In [37]:
fairSpread

-0.00779677853042454

In [43]:
tolerance = 1.0e-13

In [44]:
assetSwap2 = ql.AssetSwap(payFixedRate,
                         bond, fairCleanPrice,
                         variables.iborIndex, variables.spread,
                         ql.Schedule(),
                         variables.iborIndex.dayCounter(),
                         isPar)

In [45]:
assetSwap2.setPricingEngine(swapEngine)

In [47]:
assetSwap2.NPV()

2.842170943040401e-14

In [48]:
if (abs(assetSwap2.NPV())>tolerance):
    print("FAIL: par asset swap fair clean price doesn't zero the NPV")

In [49]:
assetSwap2.fairCleanPrice()

82.9194836716913

In [52]:
if (abs(assetSwap2.fairCleanPrice() - fairCleanPrice)>tolerance):
    print("FAIL: par asset swap fair clean price doesn't equal input")

In [53]:
assetSwap2.fairSpread()

-1.8350942472453456e-17

In [54]:
if (abs(assetSwap2.fairSpread() - variables.spread)>tolerance):
    print("FAIL: par asset swap fair spread doesn't equal input spread")

In [55]:
assetSwap3 = ql.AssetSwap(payFixedRate,
                         bond, bondPrice,
                         variables.iborIndex, fairSpread,
                         ql.Schedule(),
                         variables.iborIndex.dayCounter(),
                         isPar)

In [56]:
assetSwap3.setPricingEngine(swapEngine)

In [57]:
if (abs(assetSwap3.NPV())>tolerance):
    print("FAIL: par asset swap fair spread doesn't zero the NPV")

In [58]:
if (abs(assetSwap3.fairCleanPrice() - bondPrice)>tolerance):
    print("FAIL: par asset swap fair clean price doesn't equal input")

In [59]:
if (abs(assetSwap3.fairSpread() - fairSpread)>tolerance):
    print("FAIL: par asset swap fair spread doesn't equal input spread")

In [60]:
# let's change the npv date